In [97]:
import numpy as np
import pandas as pd
import math
from sklearn.impute import KNNImputer

In [2]:
def pairwise_dist(x, y):  # [5 pts]
    """
        Args:
            x: N x D numpy array
            y: M x D numpy array
        Return:
                dist: N x M array, where dist2[i, j] is the euclidean distance between
                x[i, :] and y[j, :]
    """
    #raise NotImplementedError
    return np.linalg.norm(x[:, None, :] - y[None, :, :], axis=-1)

In [90]:
cities = pd.read_csv("data/ds2.csv")
cities = cities.rename(columns = {'Unnamed: 0':'city_id'})
cities

,city_id,State Abbreviation,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,City,Average House Price 2020/2021,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",...,Walk,OtherTransp,WorkAtHome,MeanCommute,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,Population
0,0,NY,-0.136317,0.851226,0.851226,0.855108,New York,1.045961,1.230131,1.230131,...,1.627811,-0.553042,-1.312925,3.828220,-0.203596,0.226288,0.006249,-0.591108,-0.016201,23.673822
1,1,CA,0.737962,0.014144,0.014144,1.139948,Los Angeles,1.393001,1.882940,1.882940,...,0.092612,0.256218,0.250712,1.128201,-0.239062,-0.512411,1.786000,0.071967,0.417355,10.737820
2,2,IL,0.457552,0.224546,0.224546,0.427849,Chicago,-0.363198,-0.868798,-0.868798,...,0.845506,0.347146,-0.302575,1.687189,0.671963,-0.320016,-0.911262,-0.591108,1.564181,7.403277
3,3,TX,-0.416098,-0.755247,-0.755247,-1.263387,Houston,-0.588687,-0.388227,-0.388227,...,-0.560289,-0.034752,-0.771666,0.529134,0.669746,-0.873447,0.359989,-0.093802,-0.132749,5.907363
4,4,PA,-0.042637,1.850164,1.850164,-1.263387,Philadelphia,-0.602243,0.856306,0.856306,...,3.374761,1.256428,-0.807750,1.533204,0.654230,-0.168001,-1.204202,-1.088414,2.240155,4.068471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,945,OR,0.711106,1.890214,1.890214,0.694886,Keizer,-0.094343,0.348842,0.348842,...,0.404357,-0.162051,-0.019918,-0.540326,-0.773263,0.934109,-0.231419,-0.259570,-0.188691,-0.269062
946,946,UT,1.353025,2.799419,2.799419,-1.263387,Spanish Fork,0.058636,-0.225232,-0.225232,...,0.439649,0.210754,1.080642,-0.903141,0.654230,-0.405525,-0.695702,0.237736,-1.246941,-0.269371
947,947,WI,-0.343854,0.778758,0.778758,-1.263387,Beloit,-0.901212,NaN,NaN,...,0.086730,-0.653063,-0.819778,-0.620483,0.878107,-0.597919,-0.773082,-0.756876,0.049065,-0.269565
948,948,FL,-1.326232,-0.654708,-0.654708,-0.284251,Panama City,-0.626830,-0.485019,-0.485019,...,-0.648519,0.110733,-1.018240,-0.637358,-0.427473,0.508942,-0.159566,1.232348,-0.244634,-0.269597


In [134]:
data_us = pd.read_csv('data/uscities.csv')

data_us = data_us[['city', 'state_id', 'state_name', 'county_name', 'lat', 'lng']]
data_us['city_name_id'] = data_us.apply(lambda x: x['city'] + ", " + x['state_name'] + ", " + x['county_name'] + " County", axis = 1)
data_us['city_state_id'] = data_us.apply(lambda x: x['city'] + ", " + x['state_name'], axis = 1)

data_cities = cities.copy()
data_cities['city_name_id'] = data_cities.apply(lambda x: x['City'] + ", " + x['State'] + ", " + x['County'], axis = 1)
data_cities['city_state_id'] = data_cities.apply(lambda x: x['City'] + ", " + x['State'], axis = 1)


city_all = data_cities.merge(data_us, on = 'city_state_id', how = 'left')
city_all = city_all.rename(columns = {'city_state_id_x':'city_state'})
# city_all = city_all[~city_all['lat'].isna()]
# city_all = city_all[~city_all['lng'].isna()]
city_all = city_all.drop(columns = ['city_id', 'city_name_id_y', 'city_name_id_x', 'city_state_id'])
city_all = city_all.reset_index(drop = True)
city_all = city_all.reset_index()
city_all = city_all.rename(columns = {'index':'city_id'})
city_all['city_state'] = data_cities['city_state_id']
city_all.to_csv('data/city_all.csv', index = False)
city_all.shape

(950, 57)

Missing Cities: (Inputted by hand in city_all.csv)

city_id, City, St Abbrev, State, lat, lng
674, Milford, CT, Connecticut, 41.2307, -73.0640 
732, Methuen, MA, Massachusetts, 42.7262, -71.1909

In [146]:
city_all[city_all['lat'].isna()]

,city_id,State Abbreviation,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,City,Average House Price 2020/2021,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",...,FamilyWork,Unemployment,Population,city,state_id,state_name,county_name,lat,lng,city_state
674,674,CT,0.798253,-0.826907,-0.826907,0.783898,Milford,-0.147216,-0.808543,-0.808543,...,1.232348,0.645788,-0.227348,NaN,NaN,NaN,NaN,NaN,NaN,"Milford, Connecticut"
732,732,MA,1.494028,0.808887,0.808887,1.228960,Methuen,0.145010,0.098806,0.098806,...,0.237736,-0.435772,-0.236303,NaN,NaN,NaN,NaN,NaN,NaN,"Methuen, Massachusetts"


In [147]:
meta_columns = ['city_id', 'State', 'County', 'State Abbreviation', 'city_state', 'city', 'City', 'state_id', 'county_name', 'state_name', 'lat', 'lng']

new_city_all = pd.read_csv('data/new_city_all.csv')
new_city_all.drop(columns = meta_columns).columns
new_city_all[new_city_all['lat'].isna()]

,city_id,State Abbreviation,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,City,Average House Price 2020/2021,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",...,FamilyWork,Unemployment,Population,city,state_id,state_name,county_name,lat,lng,city_state


In [148]:
city_metadata = new_city_all[meta_columns]
#city_metadata.to_csv('data/city_metadata.csv', index = False)


In [149]:
# def id_to_city(city_id):
#     return pd.read_csv('data/city_metadata.csv').iloc[[city_id]]
# id_to_city(10)

,city_id,State,County,State Abbreviation,city_state,city,City,state_id,county_name,state_name,lat,lng
10,10,Texas,Travis County,TX,"Austin, Texas",Austin,Austin,TX,Travis,Texas,30.3005,-97.7522


In [152]:
city_df = new_city_all.drop(columns = meta_columns)
knn_imputer = KNNImputer(n_neighbors = 5)
city_df = knn_imputer.fit_transform(city_df)
city_np = np.array(city_df)
city_np.shape

(950, 45)

In [153]:
city_np

array([[-1.36317125e-01,  8.51225950e-01,  8.51225950e-01, ...,
        -5.91107781e-01, -1.62011840e-02,  2.36738216e+01],
       [ 7.37961649e-01,  1.41441490e-02,  1.41441490e-02, ...,
         7.19670230e-02,  4.17354867e-01,  1.07378196e+01],
       [ 4.57551781e-01,  2.24546057e-01,  2.24546057e-01, ...,
        -5.91107781e-01,  1.56418055e+00,  7.40327713e+00],
       ...,
       [-3.43854299e-01,  7.78758422e-01,  7.78758422e-01, ...,
        -7.56876483e-01,  4.90653190e-02, -2.69565082e-01],
       [-1.32623241e+00, -6.54708389e-01, -6.54708389e-01, ...,
         1.23234793e+00, -2.44633941e-01, -2.69596553e-01],
       [ 8.30974821e-01, -3.05575666e+00, -3.05575666e+00, ...,
        -3.00415771e-01,  6.71036275e-01,  1.30142283e-01]])

In [154]:
cities_dist = None
cities_nn = None
if True:
    cities_dist = pairwise_dist(city_np, city_np) #.tofile('data/pairwise_city_dist.csv', sep = ',')
    #np.savetxt("data/city_pairwise_dist.csv", cities_dist, delimiter=",")
    temp = pd.DataFrame(cities_dist)
    #temp.to_csv("data/city_distances.csv", index = False)
    cities_nn = pd.DataFrame(np.argsort(cities_dist))
    #np.savetxt("data/city_nn_ranked.csv", cities_nn, delimiter=",")
    #cities_nn.to_csv("data/city_nn.csv", index = False)
    cities_dist = pd.DataFrame(np.sort(cities_dist, axis = 1))
    del temp
else:
    cities_dist = np.array(pd.read_csv("data/city_distances.csv"))
    cities_nn = np.array(pd.read_csv("data/city_nn.csv"))

In [155]:
np.sum(cities_nn, axis = 1)

0      450775
1      450775
2      450775
3      450775
4      450775
        ...  
945    450775
946    450775
947    450775
948    450775
949    450775
Length: 950, dtype: int64

In [156]:
def get_nn(city_id, n = 5):
    """
        Args:
            city_id: int
            n: int
        Return:
            nearest_neighbors: (n,) ndarray, where nearest_neighbors[i] is the city_id of the i'th most similar city
    """

    #Don't include the first column since it will always be itself (distances between a city and itself is 0 miles)
    dta = cities_nn.iloc[city_id, 1:n+1]

    return dta
    
get_nn(25, 5)

1     44
2     16
3    604
4     75
5    179
Name: 25, dtype: int64

In [157]:
cities_nn.head()

,0,1,2,3,4,5,6,7,8,9,...,940,941,942,943,944,945,946,947,948,949
0,0,1,3,2,5,6,8,7,4,13,...,747,687,407,741,708,759,901,466,51,772
1,1,3,5,2,6,8,7,34,20,16,...,536,407,305,708,741,759,901,466,51,772
2,2,5,4,8,7,410,351,502,416,408,...,575,22,901,108,487,305,466,772,51,0
3,3,6,8,1,5,19,2,11,16,21,...,924,638,644,622,487,22,0,466,772,51
4,4,62,23,508,57,924,644,67,351,603,...,575,108,305,487,3,466,1,772,51,0


In [158]:
city_metadata.columns

Index(['city_id', 'State', 'County', 'State Abbreviation', 'city_state',
       'city', 'City', 'state_id', 'county_name', 'state_name', 'lat', 'lng'],
      dtype='object')

In [159]:
nearest_neighbors = pd.concat([city_metadata, cities_nn.iloc[:, 0:21]], axis = 1)

nearest_neighbors['city_county'] = nearest_neighbors.apply(lambda x: x['City'] + ', ' + x['County'], axis = 1)
nearest_neighbors['city_st_abbr'] = nearest_neighbors.apply(lambda x: x['City'] + ', ' + x['State Abbreviation'], axis = 1)

#nearest_neighbors.to_csv('data/nearest_neighbors.csv', index = False)
nearest_neighbors

,city_id,State,County,State Abbreviation,city_state,city,City,state_id,county_name,state_name,...,13,14,15,16,17,18,19,20,city_county,city_st_abbr
0,0,New York,Queens County,NY,"New York, New York",New York,New York,NY,Queens,New York,...,10,12,23,11,17,29,9,14,"New York, Queens County","New York, NY"
1,1,California,Los Angeles County,CA,"Los Angeles, California",Los Angeles,Los Angeles,CA,Los Angeles,California,...,137,12,279,223,166,107,140,9,"Los Angeles, Los Angeles County","Los Angeles, CA"
2,2,Illinois,Cook County,IL,"Chicago, Illinois",Chicago,Chicago,IL,Cook,Illinois,...,599,580,631,12,664,668,14,865,"Chicago, Cook County","Chicago, IL"
3,3,Texas,Harris County,TX,"Houston, Texas",Houston,Houston,TX,Harris,Texas,...,14,29,35,30,25,7,44,20,"Houston, Harris County","Houston, TX"
4,4,Pennsylvania,Philadelphia County,PA,"Philadelphia, Pennsylvania",Philadelphia,Philadelphia,PA,Philadelphia,Pennsylvania,...,750,2,577,502,408,664,599,631,"Philadelphia, Philadelphia County","Philadelphia, PA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,945,Oregon,Marion County,OR,"Keizer, Oregon",Keizer,Keizer,OR,Marion,Oregon,...,558,614,855,293,520,308,690,635,"Keizer, Marion County","Keizer, OR"
946,946,Utah,Utah County,UT,"Spanish Fork, Utah",Spanish Fork,Spanish Fork,UT,Utah,Utah,...,723,922,334,281,296,693,229,945,"Spanish Fork, Utah County","Spanish Fork, UT"
947,947,Wisconsin,Rock County,WI,"Beloit, Wisconsin",Beloit,Beloit,WI,Rock,Wisconsin,...,601,69,844,858,252,839,205,268,"Beloit, Rock County","Beloit, WI"
948,948,Florida,Bay County,FL,"Panama City, Florida",Panama City,Panama City,FL,Bay,Florida,...,46,184,905,570,604,685,208,724,"Panama City, Bay County","Panama City, FL"


In [160]:
cities_dist.iloc[0, :4]

0     0.000000
1    20.531299
2    23.693831
3    26.880252
Name: 0, dtype: float64

In [161]:
cities_dist_short = cities_dist.iloc[:, :21].reset_index().rename(columns={'index':'city_id'})
nn = nearest_neighbors.copy()
nn = nn.merge(cities_dist_short, on = 'city_id')


In [162]:
nn

,city_id,State,County,State Abbreviation,city_state,city,City,state_id,county_name,state_name,...,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y
0,0,New York,Queens County,NY,"New York, New York",New York,New York,NY,Queens,New York,...,32.113133,32.198062,32.274623,32.361472,32.459205,32.483711,32.523964,32.529452,32.552359,32.627523
1,1,California,Los Angeles County,CA,"Los Angeles, California",Los Angeles,Los Angeles,CA,Los Angeles,California,...,18.087110,18.156403,18.223295,18.306393,18.346822,18.349147,18.357198,18.365909,18.387377,18.390654
2,2,Illinois,Cook County,IL,"Chicago, Illinois",Chicago,Chicago,IL,Cook,Illinois,...,9.174730,9.177440,9.187375,9.190646,9.191542,9.193015,9.193264,9.197988,9.222041,9.229239
3,3,Texas,Harris County,TX,"Houston, Texas",Houston,Houston,TX,Harris,Texas,...,14.183360,14.236291,14.292241,14.330283,14.497089,14.703776,14.712995,14.772326,15.427702,15.443429
4,4,Pennsylvania,Philadelphia County,PA,"Philadelphia, Pennsylvania",Philadelphia,Philadelphia,PA,Philadelphia,Pennsylvania,...,8.545097,8.555570,8.580999,8.584432,8.601088,8.625340,8.629914,8.638247,8.657278,8.658081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,945,Oregon,Marion County,OR,"Keizer, Oregon",Keizer,Keizer,OR,Marion,Oregon,...,4.747227,4.818126,4.820007,4.876361,4.946549,5.005535,5.009038,5.012765,5.024819,5.073852
946,946,Utah,Utah County,UT,"Spanish Fork, Utah",Spanish Fork,Spanish Fork,UT,Utah,Utah,...,3.583359,3.687132,3.766692,4.456638,4.540016,4.568867,4.659577,4.742371,4.974955,5.103302
947,947,Wisconsin,Rock County,WI,"Beloit, Wisconsin",Beloit,Beloit,WI,Rock,Wisconsin,...,3.286537,3.410440,3.437186,3.450328,3.492244,3.501185,3.501470,3.537589,3.571245,3.584679
948,948,Florida,Bay County,FL,"Panama City, Florida",Panama City,Panama City,FL,Bay,Florida,...,4.086590,4.264292,4.302846,4.310780,4.316202,4.354219,4.371483,4.400364,4.445705,4.462225


In [164]:
nn.to_csv('data/nearest_neighbors.csv', index = False)
pd.read_csv('data/nearest_neighbors.csv').columns

Index(['city_id', 'State', 'County', 'State Abbreviation', 'city_state',
       'city', 'City', 'state_id', 'county_name', 'state_name', 'lat', 'lng',
       '0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x',
       '10_x', '11_x', '12_x', '13_x', '14_x', '15_x', '16_x', '17_x', '18_x',
       '19_x', '20_x', 'city_county', 'city_st_abbr', '0_y', '1_y', '2_y',
       '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y', '12_y',
       '13_y', '14_y', '15_y', '16_y', '17_y', '18_y', '19_y', '20_y'],
      dtype='object')